In [ ]:

# Dataset Classification Model Training
# This notebook was generated to train a MobileNetV2 model on your dataset.
# Follow the steps below to upload the dataset and run the training.

# Step 1: Upload the dataset zip file
from google.colab import files
print("Please upload your dataset zip file (archive 2.zip):")
uploaded = files.upload()


In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNetV2
import numpy as np
import os
import zipfile
import shutil
import warnings
warnings.filterwarnings('ignore')

# Step 2: Extract the uploaded zip file
dataset_dir = '/content/dataset'
if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

zip_path = '/content/archive 2.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

# Clean up zip file
os.remove(zip_path)

# Step 3: Detect classes and organize dataset into train and validation directories
base_dir = dataset_dir  # Assume dataset is extracted directly into dataset_dir
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')

# Detect class directories
classes = [d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))]
num_classes = len(classes)

# Create train and validation directories
if not os.path.exists(train_dir):
    for cls in classes:
        os.makedirs(os.path.join(train_dir, cls))
        os.makedirs(os.path.join(val_dir, cls))

    # Split images: 80% train, 20% validation
    for cls in classes:
        cls_dir = os.path.join(base_dir, cls)
        images = [f for f in os.listdir(cls_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]
        np.random.seed(42)
        np.random.shuffle(images)
        train_size = int(0.8 * len(images))

        for img in images[:train_size]:
            shutil.move(os.path.join(cls_dir, img), os.path.join(train_dir, cls, img))
        for img in images[train_size:]:
            shutil.move(os.path.join(cls_dir, img), os.path.join(val_dir, cls, img))

# Step 4: Load and preprocess the dataset
IMG_SIZE = 224  # MobileNetV2 expects 224x224 images
BATCH_SIZE = 32

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Step 5: Build the CNN model using MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))
base_model.trainable = False  # Freeze the base model

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

# Step 6: Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Step 7: Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

# Step 8: Evaluate the model
val_loss, val_accuracy = model.evaluate(validation_generator, verbose=0)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

# Ensure accuracy is at least 90%
if val_accuracy >= 0.90:
    print("Achieved at least 90% accuracy!")
else:
    print("Accuracy below 90%. Consider fine-tuning MobileNetV2 or increasing epochs.")

# Step 9: Save the model
model.save('/content/dataset_classification_cnn.h5')
print("Model saved as '/content/dataset_classification_cnn.h5'")

# Optional: Save to Google Drive
from google.colab import drive
drive.mount('/content/drive')
shutil.copy('/content/dataset_classification_cnn.h5', '/content/drive/MyDrive/dataset_classification_cnn.h5')
print("Model copied to Google Drive at '/content/drive/MyDrive/dataset_classification_cnn.h5'")
